In [2]:
from ds1000de import DS1000DE
scope=DS1000DE("USB0")
r = scope.inst.query
w = scope.inst.write

In [4]:
def Q(*args):
    return r(":"+":".join(args)+"?")

In [5]:
Q("MEASURE", "VPP")

'2.80e-02'

In [6]:
Q("TIMEBASE", "MODE")

'MAIN'

In [7]:
Q("TIMEBASE", "SCALE")

'1.000e-06'

In [8]:
from mhs5200 import MHS5200
port = "COM12" # Serial port.
signal_gen = MHS5200(port=port)

chan1 = signal_gen.channels[0]

In [15]:
chan1.frequency = 2
chan1.amplitude = 20

In [20]:
grid_t = 12
grid_v = 8
cycles = 2

In [17]:
w(":TIMEBASE:MODE MAIN")

(21, <StatusCode.success: 0>)

In [18]:
period = 1/chan1.frequency # Seconds

In [23]:
scale_val = period*cycles/grid_t

In [24]:
w(":TIMEBASE:SCALE {}".format(scale_val))

(37, <StatusCode.success: 0>)

In [25]:
w(":CHAN2:DISP OFF")

(17, <StatusCode.success: 0>)

In [26]:
w(":CHAN2:PROB 1")

(15, <StatusCode.success: 0>)

In [27]:
w(":CHAN2:PROB 1")

(15, <StatusCode.success: 0>)

In [37]:
scale_pct = 0.8
scale = chan1.amplitude/scale_pct/grid_v
w(":CHAN1:SCALE {}".format(scale))

(20, <StatusCode.success: 0>)

In [34]:
w(":CHAN1:OFFSET 0")

(17, <StatusCode.success: 0>)

In [42]:
class Measure(object):
    def __init__(self, scope):
        self.scope = scope
        
    @property
    def inst(self):
        return self.scope.inst
    
    @property
    def _kw_(self):
        return self.__class__.__name__.upper()
    
    
    def _write_(self, cmd, arg=None):
        if arg is None:
            self.inst.write(":{}:{}".format(self._kw_, cmd))
        else:
            self.inst.write(":{}:{} {}".format(self._kw_, cmd, arg))
    
    def _query_(self, cmd):
        return self.inst.query(":{}:{}?".format(self._kw_, cmd))
    
    def clear(self):
        self._write_("clear")

In [53]:
measurements = [
    'vpp',
    'vmax',
    'vmin',
    'vamplitude',
    'vtop',
    'vbase',
    'vaverage',
    'vrms',
    'overshoot',
    'preshoot',
    'frequency',
    'risetime',
    'falltime',
    'period',
    'pwidth',
    'nwidth',
    'pdutycycle',
    'ndutycycle',
    'pdelay',
    'ndelay',
]

In [54]:
def measurement_factory(measurement):
    def measure(self):
        value_raw = self._query_(measurement)
        return float(value_raw)
    return measure

In [55]:
for measurement in measurements:
    fcn = measurement_factory(measurement)
    prop = property(fcn)
    setattr(Measure, measurement, prop)

In [56]:
m = Measure(scope)

In [57]:
m.vmax

10.4

In [58]:
m.frequency

1.99

In [60]:
m.ndutycycle

0.506

In [61]:
m.period

0.498

In [62]:
chan1.amplitude=10

In [63]:
m.vpp

10.4

In [64]:
m.vamplitude

10.0

In [65]:
signal_gen.serial.close()